In [ ]:
# ==============================================
# CÉLULA 1: DIAGNÓSTICO DE ARQUIVOS E CAMINHOS
# ==============================================
# Verifica onde estão localizados os dados e modelos
# Útil para troubleshooting de caminhos incorretos

import os

# Monta Google Drive se necessário
try:
    from google.colab import drive
    drive.mount('/content/drive')
    print("Google Drive montado com sucesso!")
except:
    print("Ambiente local detectado ou Drive já montado")

# Verifica caminhos possíveis
possible_paths = [
    "/content/drive/MyDrive/Eixo_05/dados/",
    "/content/drive/MyDrive/dados/", 
    "/content/dados/",
    "./dados/",
    "../dados/",
    "dados/",
    "/home/pellizzi/projects/eixo5_grupo2_20252/dados/"
]

print("\nVerificando caminhos possíveis:")
for path in possible_paths:
    if os.path.exists(path):
        print(f"✓ ENCONTRADO: {path}")
        if os.path.exists(path + "modelos/"):
            print(f"  └── ✓ Pasta modelos existe: {path}modelos/")
            # Lista arquivos na pasta modelos
            try:
                files = os.listdir(path + "modelos/")
                print(f"      Arquivos encontrados: {files}")
            except:
                print("      Erro ao listar arquivos")
        else:
            print(f"  └── ✗ Pasta modelos NÃO existe")
    else:
        print(f"✗ NÃO EXISTE: {path}")

# Lista arquivos no diretório atual para orientação
print(f"\nArquivos no diretório atual:")
try:
    current_files = os.listdir(".")
    for f in current_files:
        print(f"  {f}")
except:

    print("Erro ao listar diretório atual")print("="*50)

print("Execute esta célula para identificar o caminho correto!")
print("\n" + "="*50)

In [ ]:
# ================================================
# CÉLULA 2: CONFIGURAÇÃO E DETECÇÃO DE CAMINHOS
# ================================================
# Inicializa Spark e detecta automaticamente onde estão os dados
# Verifica se modelos pré-treinados estão disponíveis

import os
from pyspark.sql import SparkSession

spark = SparkSession.builder.getOrCreate()

# Detecta automaticamente o caminho correto dos dados
possible_base_paths = [
    "/content/drive/MyDrive/Eixo_05/dados/",
    "/content/drive/MyDrive/dados/", 
    "/content/dados/",
    "./dados/",
    "../dados/",
    "dados/",
    "/home/pellizzi/projects/eixo5_grupo2_20252/dados/"
]

base_path = None
for path in possible_base_paths:
    if os.path.exists(path):
        base_path = path
        break

if base_path is None:
    raise Exception("Diretório de dados não encontrado. Verifique se os dados estão disponíveis.")

print(f"Usando caminho: {base_path}")

models_path = base_path + "modelos/"
if not os.path.exists(models_path):
    raise Exception(f"Modelos não encontrados em {models_path}. Execute primeiro 'aprendizado_maquina.ipynb'")

metadata_path = f"{models_path}training_metadata.json"

if not os.path.exists(metadata_path):print("Modelos encontrados!")

    raise Exception(f"Metadados não encontrados em {metadata_path}. Execute o treinamento primeiro.")

OK -> Dados carregados: 50000 50000 50000


In [ ]:
# ===============================================
# CÉLULA 3: CARREGAMENTO DE MODELOS E DATASETS
# ===============================================
# Carrega metadados do treinamento e modelos salvos
# Prepara dataset com a melhor featurização identificada

import json
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.classification import LogisticRegressionModel, LinearSVCModel, OneVsRestModel

# Carrega metadados
with open(metadata_path, 'r') as f:
    metadata = json.load(f)

best_classifier = metadata['best_classifier']
best_featurization = metadata['best_featurization']
best_accuracy = metadata['best_accuracy']

print(f"Melhor modelo: {best_classifier} com {best_featurization} ({best_accuracy:.2f}%)")

# Carrega dataset
dataset_mapping = {
    "HTFfeaturizedData": "HTFfeaturizedData",
    "TFIDFfeaturizedData": "TFIDFfeaturizedData", 
    "W2VfeaturizedData": "W2VfeaturizedData"
}

optimal_dataset_path = base_path + dataset_mapping[best_featurization]
ds = spark.read.parquet(optimal_dataset_path)
train, test = ds.randomSplit([0.8, 0.2], seed=42)

print(f"Dataset: {ds.count():,} registros | Treino: {train.count():,} | Teste: {test.count():,}")

# Carrega modelos
lr_model_path = f"{models_path}lr_{best_featurization}"
svc_model_path = f"{models_path}svc_{best_featurization}"

lr_model = LogisticRegressionModel.load(lr_model_path)

try:print("Modelos carregados com sucesso!")

    svc_model = OneVsRestModel.load(svc_model_path)

except:    svc_model = LinearSVCModel.load(svc_model_path)

In [ ]:
# ========================================
# CÉLULA 4: AVALIAÇÃO E ANÁLISE INDIVIDUAL
# ========================================
# Avalia cada modelo carregado com métricas detalhadas
# Gera matrizes de confusão para análise visual

import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

def evaluate_model(name, model, test):
    preds = model.transform(test).cache()
    
    eval_acc = MulticlassClassificationEvaluator(metricName="accuracy")
    eval_f1 = MulticlassClassificationEvaluator(metricName="f1")
    eval_precision = MulticlassClassificationEvaluator(metricName="weightedPrecision")
    eval_recall = MulticlassClassificationEvaluator(metricName="weightedRecall")
    
    accuracy = eval_acc.evaluate(preds)
    f1_score = eval_f1.evaluate(preds)
    precision = eval_precision.evaluate(preds)
    recall = eval_recall.evaluate(preds)
    
    print(f"{name}: Acurácia {accuracy:.3f} | F1 {f1_score:.3f} | Precisão {precision:.3f} | Recall {recall:.3f}")
    
    # Matriz de confusão
    confusion_data = preds.groupBy("label", "prediction").count().collect()
    confusion_matrix = [[0, 0], [0, 0]]
    for row in confusion_data:
        real = int(row['label'])
        pred = int(row['prediction'])
        count = row['count']
        confusion_matrix[real][pred] = count
    
    plt.figure(figsize=(6, 4))
    sns.heatmap(confusion_matrix, annot=True, fmt='d', cmap='Blues',
                xticklabels=['Negativo', 'Positivo'],
                yticklabels=['Negativo', 'Positivo'])
    plt.title(f'Matriz de Confusão - {name}')
    plt.show()
    
    return {
        'model': name,
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1_score': f1_score
    }

# Análise dos modelos
print("Avaliando modelos...")
modelos = [("Logistic Regression", lr_model), ("Linear SVC", svc_model)]
resultados = []

print("Análise concluída!")

for nome, modelo in modelos:

    resultado = evaluate_model(nome, modelo, test)    resultados.append(resultado)


=== LogisticRegression (TFIDFfeaturizedData) ===
Acurácia : 89.16%
Taxa erro: 10.84%
F1-score : 0.8916
Matriz de confusão (label x prediction):
+-----+----------+-----+
|label|prediction|count|
+-----+----------+-----+
|0.0  |0.0       |4312 |
|0.0  |1.0       |583  |
|1.0  |0.0       |490  |
|1.0  |1.0       |4516 |
+-----+----------+-----+


=== LinearSVC (TFIDFfeaturizedData) ===
Acurácia : 90.27%
Taxa erro: 9.73%
F1-score : 0.9027
Matriz de confusão (label x prediction):
+-----+----------+-----+
|label|prediction|count|
+-----+----------+-----+
|0.0  |0.0       |4354 |
|0.0  |1.0       |541  |
|1.0  |0.0       |422  |
|1.0  |1.0       |4584 |
+-----+----------+-----+



In [ ]:
# =====================================
# CÉLULA 5: COMPARAÇÃO ENTRE MODELOS
# =====================================
# Compara performance dos modelos lado a lado
# Gera gráficos comparativos de métricas

df_comparison = pd.DataFrame(resultados)

print("\nComparação:")
for _, row in df_comparison.iterrows():
    print(f"{row['model']}: Acurácia {row['accuracy']:.3f} | F1 {row['f1_score']:.3f}")

# Gráfico comparativo simples
plt.figure(figsize=(8, 5))
metrics = ['accuracy', 'f1_score']
for i, metric in enumerate(metrics):
    plt.subplot(1, 2, i+1)
    models = df_comparison['model']
    values = df_comparison[metric]
    plt.bar(models, values, color=['skyblue', 'lightcoral'])
    plt.title(f'{metric.capitalize()}')

    plt.ylim(0, 1)plt.show()

    plt.xticks(rotation=45)plt.tight_layout()


In [ ]:
# ========================================
# CÉLULA 6: RESULTADO FINAL E VALIDAÇÃO
# ========================================
# Identifica o melhor modelo e valida com dados de treinamento
# Apresenta conclusão do pipeline otimizado

best_model = df_comparison.loc[df_comparison['accuracy'].idxmax()]

print(f"\nMelhor modelo: {best_model['model']}")
print(f"Acurácia: {best_model['accuracy']:.3f} ({best_model['accuracy']*100:.1f}%)")
print(f"F1-Score: {best_model['f1_score']:.3f}")

# Validação com treinamento
if best_classifier in best_model['model']:

    print("Resultado consistente com o treinamento!")print("\nAnálise concluída. Pipeline otimizado funcionando!")

else:
    print("Diferença detectada nos resultados.")